<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/re_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

df_small = df[['Player', 'Bye', 'POS', 'AVG']].head(250).copy()
df_small['Bye'] = df_small['Bye'].apply(extract_numbers)
df_small['Bye'] = df_small['Bye'].fillna(0).astype(int)

df_small.info()
print(df_small.value_counts('POS'))
print(df_small.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  250 non-null    object 
 1   Bye     250 non-null    int64  
 2   POS     250 non-null    object 
 3   AVG     250 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 7.9+ KB
POS
WR     77
RB     66
QB     31
DST    28
TE     28
K      20
Name: count, dtype: int64
                 Player  Bye POS   AVG
0   Christian McCaffrey    9  RB   1.0
1           CeeDee Lamb    7  WR   2.6
2           Tyreek Hill    6  WR   3.2
3        Bijan Robinson   12  RB   5.0
4           Breece Hall   12  RB   5.4
5     Amon-Ra St. Brown    5  WR   6.2
6         Ja'Marr Chase   12  WR   6.6
7      Justin Jefferson    6  WR   7.0
8        Saquon Barkley    5  RB   9.2
9            A.J. Brown    5  WR  10.2
10      Jonathan Taylor   14  RB  10.4
11       Garrett Wilson   12  WR  12.4
12         Jahm

In [3]:
# Anzahl der Wochen
number_of_weeks = 17

# Wochen-Spaltennamen
weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# Skalierungsfunktion
def final_projection_base(avg, pos, max_val=22, min_val=7, k=50, c=1.5):
    base = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
    if pos == "QB":
        base += 4  # QB-Bonus
    return base

final_projections = []
for _, row in df_small.iterrows():
    base_score = final_projection_base(row['AVG'], row['POS'])

    # Erstellen der weekly projections
    weekly_proj = []
    for week in range(number_of_weeks):
        # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
        if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
            weekly_proj.append(0.0)  # Projektion auf 0 setzen
        else:
            weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

    final_projections.append(weekly_proj)

# Projektionen in DataFrame einfügen
f = df_small[["Player", "Bye", "POS", "AVG"]].copy()
for i, col in enumerate(weekly_columns):
    f[col] = [proj[i] for proj in final_projections]

# Zeige eine zufällige Stichprobe der ersten 25 Zeilen
f.head(25)


,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17
0,Christian McCaffrey,9,RB,1.0,21.883941,22.654547,20.815160,25.171561,18.938098,22.316645,...,21.036601,0.000000,24.295338,23.294477,22.692065,26.962897,21.336309,21.068680,18.866533,18.428070
1,CeeDee Lamb,7,WR,2.6,24.151651,21.318112,21.101154,22.651551,24.425087,25.333047,...,19.706212,21.766677,20.712996,22.279408,21.312576,25.939188,25.663795,21.456597,18.756094,21.520334
2,Tyreek Hill,6,WR,3.2,19.666306,21.511501,16.826665,21.808360,22.339030,0.000000,...,21.141379,20.002039,21.217313,20.337654,18.916461,19.620733,20.183645,20.075558,26.664996,24.138591
3,Bijan Robinson,12,RB,5.0,19.204148,17.784220,17.562386,20.724167,22.377590,17.374568,...,19.357626,24.249271,19.208613,19.648685,0.000000,20.394899,24.357438,22.819246,24.046460,23.275679
4,Breece Hall,12,RB,5.4,21.678077,22.778682,23.077592,23.203857,19.901376,22.850923,...,22.234657,21.551728,24.801725,19.748939,0.000000,19.499318,21.161232,24.261555,20.424001,21.160386
5,Amon-Ra St. Brown,5,WR,6.2,21.622690,20.548012,24.929524,20.254637,0.000000,19.914401,...,19.080191,17.809685,23.394545,21.207997,18.017554,25.866569,19.918069,21.064511,22.407419,20.433606
6,Ja'Marr Chase,12,WR,6.6,18.863471,20.941962,21.659149,20.738767,22.242407,24.099313,...,20.735183,19.932496,21.197064,22.276027,0.000000,21.418215,22.050329,23.266863,18.282309,19.087275
7,Justin Jefferson,6,WR,7.0,24.776217,26.698669,21.814730,20.439286,20.018035,0.000000,...,22.951033,23.399096,21.209249,17.662970,21.269132,17.648396,20.206395,20.502581,23.769811,22.591504
8,Saquon Barkley,5,RB,9.2,16.148465,22.172816,19.602059,20.710486,0.000000,21.993708,...,20.848568,22.393197,18.074003,18.558461,21.507100,19.107049,17.528938,22.984121,19.342463,24.118151
9,A.J. Brown,5,WR,10.2,18.273963,19.487161,18.442102,18.158813,0.000000,22.532786,...,22.415586,24.135893,15.960014,21.148674,23.157257,21.554455,16.979583,20.348809,20.874789,21.287101


In [11]:
week_cols = [col for col in f.columns if col.startswith("Week_")]
f_dict = {
    row['Player']: {int(week.replace("Week_", "")): row[week] for week in week_cols}
    for _, row in f.iterrows()
}
print(f_dict['Christian McCaffrey'][9])
print(f_dict['CeeDee Lamb'][17])
list(f_dict.items())[:3]

0.0
21.520334371290488


[('Christian McCaffrey',
  {1: 21.883940729640422,
   2: 22.6545466178222,
   3: 20.815159655572277,
   4: 25.171561387043656,
   5: 18.93809766565225,
   6: 22.31664454527242,
   7: 21.923543808481238,
   8: 21.036600905846406,
   9: 0.0,
   10: 24.295337709327164,
   11: 23.294476542488056,
   12: 22.692064898794577,
   13: 26.962897117957482,
   14: 21.336308536230163,
   15: 21.068679591922223,
   16: 18.866532999329397,
   17: 18.428069862281788}),
 ('CeeDee Lamb',
  {1: 24.15165109646459,
   2: 21.318111951060587,
   3: 21.10115427574404,
   4: 22.651550723660414,
   5: 24.425086856709516,
   6: 25.333047461152013,
   7: 0.0,
   8: 19.706212169725536,
   9: 21.766676601950373,
   10: 20.712996293508454,
   11: 22.279408145741314,
   12: 21.31257584695556,
   13: 25.93918822589793,
   14: 25.663794996079933,
   15: 21.456596933568267,
   16: 18.756093569982653,
   17: 21.520334371290488}),
 ('Tyreek Hill',
  {1: 19.66630636310859,
   2: 21.511500992258917,
   3: 16.826664912522542

In [7]:
!pip install mip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


In [8]:
import pandas as pd
from mip import Model, BINARY, maximize, xsum

# 1. Spieler-Liste und Positionszuordnung
players = df_small['Player'].tolist()
pos = dict(zip(df_small['Player'], df_small['POS']))

# 3. Weitere Parameter
weeks = list(range(1, 18))
positions = {"QB", "RB", "WR", "TE", "K", "DST"}
gamma = {"QB": 2, "RB": 5, "WR": 6, "TE": 2, "K": 1, "DST": 2}
pos_limit = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DST": 1}
beta = {t: 100.0 for t in weeks}

# 4. Beispielhafte Listen für bereits gedraftete Spieler (hier leer)
dm_players = []
opp_players = []

# 5. Initialisierung des Modells
model = Model("FantasyFootball", sense=maximize)

# Entscheidungsvariablen
y = {i: model.add_var(name=f"y_{i}", var_type=BINARY) for i in players} #
x = {(i, t): model.add_var(name=f"x_{i}_{t}") for i in players for t in weeks}
z = {t: model.add_var(name=f"z_{t}", var_type=BINARY) for t in weeks}

# Zielfunktion
model.objective = (
    1.0 * xsum(f_dict[i][t] * x[i, t] for i in players for t in weeks) +
    100.0 * xsum(z[t] for t in range(1, 16)) +
    150.0 * xsum(z[t] for t in range(16, 18))
)

# Constraints
for j in positions:
    # Mindestanzahl gedrafteter Spieler pro Position (1c)
    model += xsum(y[i] for i in players if pos[i] == j) >= gamma[j]
    # Positionslimits pro Woche (1d)
    for t in weeks:
        model += xsum(x[i, t] for i in players if pos[i] == j) <= pos_limit[j]

model += xsum(y[i] for i in players) <= 18

# Logische Verknüpfung Draft ↔ Aufstellung (1e)
for i in players:
    for t in weeks:
        model += x[i, t] <= y[i]

# Gewinnbedingung (1f)
for t in weeks:
    model += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in players) / beta[t]

# Bereits gedraftete Spieler (1g-1h)
for i in dm_players:
    model += y[i] == 1
for i in opp_players:
    model += y[i] == 0

# Optimierung starten
model.optimize()


<OptimizationStatus.OPTIMAL: 0>

In [9]:
drafted_players = [i for i in players if y[i].x >= 0.99]
print("Dein Team:")
for player in drafted_players:
    print(player)

# Zielfunktionswert (Gesamtpunkte)
print("Zielfunktionswert:", model.objective_value)

Dein Team:
Joe Burrow
Raheem Mostert
David Njoku
Devin Singletary
Zack Moss
Tyjae Spears
Brandon Aubrey
Matthew Stafford
Kansas City Chiefs
Adonai Mitchell
Xavier Legette
Dontayvion Wicks
Darnell Mooney
Braelon Allen
Ben Sinnott
Marvin Mims Jr.
Jacksonville Jaguars
Wan'Dale Robinson
Zielfunktionswert: 0.0


In [16]:
import pandas as pd
from mip import Model, BINARY, maximize, xsum

### Parameter ###
# The set of NFL players and defensive teams.
players = df_small['Player'].tolist()

# The set of positions
positions = {"QB", "RB", "WR", "TE", "K", "DST"}

# The set of weeks in the NFL regular and playoff seasons
weeks = list(range(1, 18))

# Players and their position
pos = dict(zip(df_small['Player'], df_small['POS']))

# Position Limit: The upper bound on the number of starting players for position
pos_limit = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DST": 1}

# The overall pick number of the DM’s k-th draft pick
n_k = 0

# The set of players that the DM has drafted by her k-th pick
dm_players = []

# The set of players that the Opponents have drafted by her k-th pick
opp_players = []

gamma = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "K": 1, "DST": 1}
beta = {t: 100.0 for t in weeks}

# Teams und Snake-Draft-Reihenfolge
number_of_teams = 6
teams = [f"Team {i+1}" for i in range(number_of_teams)]
num_rounds = 15

# Snake Draft Reihenfolge erzeugen
draft_order = []
for r in range(num_rounds):
    if r % 2 == 0:
        draft_order.extend(teams)
    else:
        draft_order.extend(teams[::-1])

# Initialisiere leere Roster und verfügbare Spieler
rosters = {team: [] for team in teams}
available_players = set(players)

# Initialisiere leere Draft Reihenfolge
draft_log = []


for pick_num, team in enumerate(draft_order):
    model = Model(sense=maximize, solver_name="CBC")

    # Entscheidungsvariablen
    y = {i: model.add_var(name=f"y_{i}", var_type=BINARY) for i in players} #
    x = {(i, t): model.add_var(name=f"x_{i}_{t}") for i in players for t in weeks}
    z = {t: model.add_var(name=f"z_{t}", var_type=BINARY) for t in weeks}

    # Bereits gedraftete Spieler verbieten
    for i in players:
        if i not in available_players:
            model += y[i] == 0

    # Bereits im Team befindliche Spieler erzwingen
    for i in rosters[team]:
        model += y[i] == 1

    # Zielfunktion
    model.objective = (
        xsum(f_dict[i][t] * x[i, t] for i in players for t in weeks) +
        100.0 * xsum(z[t] for t in range(1, 16)) +
        150.0 * xsum(z[t] for t in range(16, 18))
    )

    # Constraints
    for j in positions:
        model += xsum(y[i] for i in players if pos[i] == j) >= gamma[j]
        for t in weeks:
            model += xsum(x[i, t] for i in players if pos[i] == j) <= pos_limit[j]

    # Rostergröße
    model += xsum(y[i] for i in players) <= 18

    for i in players:
        for t in weeks:
            model += x[i, t] <= y[i]

    for t in weeks:
        model += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in players) / beta[t]

    # Optimierung
    status = model.optimize()

    # Spieler auswählen, der neu gedraftet wird
    picked_player = None
    for i in players:
        if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team] and i in available_players:
            picked_player = i
            break

    # if picked_player is None:
    #     print(f"Kein Spieler für Team {team} gefunden bei Pick {pick_num + 1}")
    #     break

    # Spieler zum Roster hinzufügen und aus verfügbaren Spielern entfernen
    rosters[team].append(picked_player)
    available_players.remove(picked_player)

    # Loggen
    round_num = pick_num // len(teams) + 1
    draft_log.append({"Pick": pick_num + 1, "Team": team, "Player": picked_player, "Round": round_num})

# DataFrame aus Log erzeugen
df_draft = pd.DataFrame(draft_log)
df_draft


KeyError: None